In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
#loading dataset
data=pd.read_csv("bank_data.csv")
data.head()

,age,default,balance,housing,loan,duration,campaign,pdays,previous,poutfailure,...,johousemaid,jomanagement,joretired,joself.employed,joservices,jostudent,jotechnician,jounemployed,jounknown,y
0,58,0,2143,1,0,261,1,-1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,44,0,29,1,0,151,1,-1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,33,0,2,1,1,76,1,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,47,0,1506,1,0,92,1,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33,0,1,0,0,198,1,-1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
#EDA
data.shape
data.dtypes
data.isna
data.describe()

data.groupby('y').mean()

plt.scatter(x=data['age'],y=data['y'],color="brown")
plt.show()
plt.scatter(x=data['duration'],y=data['y'],color="brown")
plt.show()
plt.scatter(x=data['balance'],y=data['y'],color="brown")
plt.show()

<bound method DataFrame.isna of        age  default  balance  housing  loan  duration  campaign  pdays  \
0       58        0     2143        1     0       261         1     -1   
1       44        0       29        1     0       151         1     -1   
2       33        0        2        1     1        76         1     -1   
3       47        0     1506        1     0        92         1     -1   
4       33        0        1        0     0       198         1     -1   
...    ...      ...      ...      ...   ...       ...       ...    ...   
45206   51        0      825        0     0       977         3     -1   
45207   71        0     1729        0     0       456         2     -1   
45208   72        0     5715        0     0      1127         5    184   
45209   57        0      668        0     0       508         4     -1   
45210   37        0     2971        0     0       361         2    188   

       previous  poutfailure  ...  johousemaid  jomanagement  joretired  \
0   

In [54]:
### Splitting the data into train and test data 
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(data,test_size = 0.3) # 30% test data

In [61]:
# Model building 
import statsmodels.formula.api as sm
data.head()
logit_model = sm.logit('y ~ age+default+balance+housing+loan+duration+campaign+pdays+previous+married+single+jotechnician',data = train_data).fit()

Optimization terminated successfully.
         Current function value: 0.276662
         Iterations 7


In [62]:
#summary
logit_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                31647
Model:                          Logit   Df Residuals:                    31634
Method:                           MLE   Df Model:                           12
Date:                Sat, 25 Apr 2020   Pseudo R-squ.:                  0.2326
Time:                        00:44:32   Log-Likelihood:                -8755.5
converged:                       True   LL-Null:                       -11410.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8362      0.122    -23.159      0.000      -3.076      -2.596
age              0.0047      0.002      2.339      0.019       0.001       0.009
default         -0.1899      0.177     -1.074      0.283      -0.536       0.157
balance       2.372e-05   5.37e-06      4.415      0.000    1.32e-05    3.42e-05
housing         -1.1966      0.043    -27.520      0.000      -1.282      -1.111
loan            -0.6490      0.066     -9.768      0.000      -0.779      -0.519
duration         0.0039   7.12e-05     54.116      0.000       0.004       0.004
campaign        -0.1213      0.011    -10.714      0.000      -0.144      -0.099
pdays            0.0031      0.000     15.977      0.000       0.003       0.003
previous         0.0722      0.009      7.855      0.000       0.054       0.090
married         -0.1518      0.064     -2.356      0.018      -0.278      -0.025
single           0.3086      0.073      4.234      0.000       0.166       0.451
jotechnician    -0.0641      0.055     -1.173      0.241      -0.171       0.043
================================================================================
"""

In [67]:

predict = logit_model.predict(pd.DataFrame(test_data[['age','default','balance','housing','loan','duration','campaign','pdays','previous','married','single','jotechnician']]))
predict



15353    0.021356
11933    0.003388
27988    0.106428
12739    0.079154
14513    0.057363
           ...   
40837    0.190662
40732    0.130281
5176     0.024148
21005    0.060110
22218    0.034061
Length: 13564, dtype: float64

In [68]:
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(test_data['y'], predict > 0.5 )
cnf_matrix

array([[11755,   217],
       [ 1250,   342]], dtype=int64)

In [70]:
accuracy = (11755+342/(11755+342+1250+217))
accuracy #11755.025213801238

11755.025213801238

In [72]:
from sklearn.metrics import roc_curve
#roc curve
y_true = test_data['y']
fpr,tpr, thresholds = roc_curve(y_true,predict)
fpr
tpr
thresholds

array([1.99999970e+00, 9.99999700e-01, 9.99993318e-01, ...,
       1.26093958e-02, 1.26079592e-02, 1.56273480e-04])